In [90]:
import pandas as pd
import re

In [92]:

df = pd.read_csv("ai_tweets.csv", encoding='ISO-8859-1')
df.head()

,Tweet,created_at
0,@cryptogems555 Invest in a currency that is de...,3/14/2025
1,While having all the hope around chatGPT lets ...,3/14/2025
2,@IGDEFI1 Join the movement towards a more sust...,3/14/2025
3,"There's a snowstorm out so naturally, I decide...",3/14/2025
4,@unclebobcrypto @Stablzone Invest in a currenc...,3/14/2025


In [93]:
df = df[~df['created_at'].astype(str).str.contains("Time", na=False)]
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df = df[df['created_at'].notna()]

df.drop_duplicates(inplace=True)

df.dropna(subset=['Tweet', 'created_at'], inplace=True)

def clean_text(text):
    text = str(text)
    text = re.sub(r"http\S+", "", text)  
    text = re.sub(r"@\w+", "", text)    
    text = re.sub(r"#\w+", "", text)     
    text = re.sub(r"[^\w\s]", "", text)  
    text = re.sub(r"\s+", " ", text)     
    return text.strip().lower()

df['cleaned_text'] = df['Tweet'].apply(clean_text)
df.drop_duplicates(subset=['cleaned_text'], inplace=True)

df['Tweet'] = df['cleaned_text']
df.drop(columns=['cleaned_text'], inplace=True)

df.head()


,Tweet,created_at
0,invest in a currency that is designed to offer...,2025-03-14
1,while having all the hope around chatgpt lets ...,2025-03-14
2,join the movement towards a more sustainable a...,2025-03-14
3,theres a snowstorm out so naturally i decided ...,2025-03-14
4,invest in a currency that offers a more secure...,2025-03-14


In [94]:
df.shape

(123746, 2)

In [96]:
na_counts = df.isna().sum()
print(na_counts)


Tweet         0
created_at    0
dtype: int64


In [100]:
duplicate_counts = df.duplicated().sum()
duplicate_counts

0

In [115]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')  

db = client['ai_tweets_db']  
collection = db['tweets']  

data_dict = df.to_dict("records")

collection.insert_many(data_dict)

print("The data has been stored in MongoDB")

The data has been stored in MongoDB
